In [1]:
import os
import comFuncs
from comFuncs import *
import comFuncs3

import string
import rarfile
import re
from multiprocessing import Pool
import time
import resource
import functools

In [2]:
try:
    ind=load_obj('datas/indexdB')
except:
    names=!find /media/some/sam/dall -type f -iname "index*2016*"
    names=names+["/media/some/sam/dall/New ebook packs for August 2016/New ebook packs INDEX 2003-2015.txt"]
    lst=[list(reversed(os.path.split(f))) for f in names]
    result = {}
    for item in lst:
        if not item[0] in result.keys():
            result[item[0]]=[]  
        result[item[0]]=result[item[0]]+ [item[1]]

    ind=[]
    for k in result.keys():
        rd=open(result[k][0]+"/"+k).readlines()
        ind=ind+ zip(rd,[result[k][0]]*len(rd),[k]*len(rd))
    ind=[idc for idc in ind if idc[0].count('\\')>0]
    save_obj(ind, 'datas/indexdB')  
len(ind)

573313

In [3]:
def removeLTdash(strg,charr):
    if len (strg)>3:
        strgg=strg.strip()
        if strgg[0]==charr:
            strgg=strgg[1:].strip()
        if strgg[-1]==charr:
            strgg=strgg[:-1].strip()
        return strgg
    return strg

In [4]:
def extractSeries(strg):
    series=re.findall('.+-(\s*\[.+\]\s*-).+.rar',strg)
    if len(series)>0:
        series=removeLTdash(series[0],'-')
        series=removeLTdash(series,'[')
        series=removeLTdash(series,']')
        return series
    return ""

In [5]:
def extractAuthor(strg):
    author=re.findall('(.+?\s*-)',strg)
    if len(author)>0:
        return removeLTdash(author[0],'-')
    return ""

In [6]:
def extractAdd(title):
    add=[]
    if title.count(')')>0 and title.count('(')>0:
        cnt=0
        while min([title.count('('),title.count(')')])>0:
        
            add.append(re.findall('.*\(\s*(.+?)\s*\)',title)[0] .strip())
        
            title=title.replace(add[-1],"")
            title=re.sub(' +',' ',title)
            cnti=0
            while title.count('()')>0:
                title=title.replace("()","").strip()
                if cnti>9:
                    return title
                cnti+=1
            #print title
            cntj=0   
            while title.count('( )'):
                title=title.replace("( )","").strip()
                if cntj>9:
                    return title,add
                cntj+=1
            if cnt>9:
                return title,add
            cnt+=1
    return title,add

In [7]:
def extractTitle(strg):
    title=re.findall('.+(\s*-.+)\(.*',strg)
    if len(title)>0:
        title=removeLTdash(title[0],'-')
        return title
    return ""

In [8]:
def extractExt(strg):
    ext=re.findall('.+(\s\(.+\)\s*).rar',strg)
    if len(ext)>0:
        return  ''.join(c for c in ext[0] if c in string.letters+string.digits+" ").strip()
    return ""

In [9]:
def extractRecord(inn):
        tmp = inn.strip().split('\\')
        rar=''.join(t for t in tmp[0] if t in string.digits+"- ")
        rar=re.sub(' +',' ',rar)
        if len(tmp)>1:
            series = extractSeries(tmp[1])
            author = extractAuthor(tmp[1])
            title, add = extractAdd(extractTitle(tmp[1]))
            exten = extractExt(tmp[1])
            strg=tmp[1].replace('.rar','')
            strg=strg.replace(author,'')
            strg=strg.replace(title,'')
            strg=strg.replace(series,'')
            strg=strg.replace(exten,'')
            for l in add:
                strg=strg.replace(l,'')
            rem=''.join(st for st in strg if st in string.letters+" '")
            rem=' '.join(rem.split())
            #print 'remaining: ',strg
            #rar folder, file,extra
            return [rar,inn,rem,author,series,title,' '.join(add),exten]
           

In [10]:
indd=zip(*ind)[0]
indd[0]

'03-01-2016 Part 1-2\\A J Bromley - [Annie Parker Mystery Short] - Welcome to Sweetwater (Christmas Cookies; Spring Fling; Shopping Spree; Slim Gym; Afternoon Delight) (retail) (azw3).rar\r\n'

In [11]:
ind[0]

('03-01-2016 Part 1-2\\A J Bromley - [Annie Parker Mystery Short] - Welcome to Sweetwater (Christmas Cookies; Spring Fling; Shopping Spree; Slim Gym; Afternoon Delight) (retail) (azw3).rar\r\n',
 '/media/some/sam/dall/New ebook packs for March 2016',
 'Index 2016-03.txt')

In [12]:
t0 = time.time()
try:
    dBB=load_obj('datas/dBB')
except:
    
    p = Pool(8)
    dBB=p.map(extractRecord, indd)
    p.terminate()
    p.close()
    dBB=filter(None,dBB)
    save_obj(dBB, 'datas/dBB') 
t1 = time.time()
total = t1-t0
print('records: ',len(dBB))
print('time: ',total)
print('resources: ',resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1000)

('records: ', 573313)
('time: ', 32.61633110046387)
('resources: ', 877)


In [13]:
def matchRec(rec,search):

    cos=[]
    for r in rec[2:]:
        if len(r)>2:
            for se in search.split(' '):
                if len(se)>1:
                    cc=get_cosine(text_to_vector(r.lower()),text_to_vector(se.lower()))
                    #print r,se
                    if cc>0.0:
                        cos.append(cc)
    return cos

In [14]:

def findrec(rec,search):
    res=matchRec(rec,search)
    if len(res)>0:
        score=sum(res)/len(search.split(' '))
        #print rec,score,res
        
        if score>.5:
            rt=rec+[score]+res
            print ' '.join( rt[2:6])
            return rt

In [15]:
def searchdb(seTerm):
    seTerm=[se.strip() for se in seTerm.split(' ')]
    allres=[list
            (filter
             (lambda x: any(xx.lower().replace("'","").count(se.strip().lower()) > 0 for xx in x[3:7]  ), dBB)) 
            for se in seTerm]
    #find common items in multiple lists
    return reduce(lambda sx, sy: filter(lambda x: x in sx,sy), ((i) for i in allres))

In [71]:
def searchindd(seTerm):
    
   
    if len(seTerm)>=2 and len(seTerm)<7:
        print "Search Term: ",' '.join(seTerm)
        allres=[list
                (filter
                 (lambda x: any(xx.lower().replace("'","").count(se.strip().lower()) > 0 
                                for xx in x.replace("_"," ").replace("-"," ").split('\\')[1].split(" ") ), indd)) 
                for se in seTerm]
        #find common items in multiple lists
        return reduce(lambda sx, sy: filter(lambda x: x in sx,sy), ((i) for i in allres))
    else:
        return []

In [39]:
search="powerone hospital"
res=searchindd(search)
res

search term:  powerone hospital


['03-13-2016 Part 2-2\\Powerone - Hell Hospital [MF] (pdf).rar\r\n']

In [18]:
def findr(name, path):
    #print name,path
    res=!find $path -type f -iname $name
    return res

In [19]:
def similarity(a,b):
    return get_cosine(text_to_vector(a.lower()),text_to_vector(b.lower()))

In [ ]:
foo="/home/some/an/"
!mkdir "$foo"
cnt=0
sor = sorted(res, key=lambda tup: tup[7])
#epub first
sorr=list(filter(lambda x: x[7].lower().count('epub') > 0, sor))+list(filter(lambda x: x[7].lower().count('epub') == 0, sor))
books=[]
for r in sorr:
 
        found=r[0]
        baseName=' '.join(ss for ss in r[3:6] if len(ss)>0).lower()
        if any(similarity(baseName,b.lower())>.75 for b in books):
            continue
        found=r[0].replace("  "," ").replace(" ","*").replace("_","*")+"*.rar"
        fls=findr(found,'/media/some/sam/dall/')
        #print found
        if len(fls)>0:
            for flss in fls:
                rf = rarfile.RarFile(flss)
                booFound=False
                for f in rf.infolist():
                    #print f.filename, f.file_size
                    if similarity(f.filename,r[1])>0.99:
                        cnt+=1
                        booFound=True
                        print cnt,"- ",f.filename,baseName
                        try:
                            rf.extract(f,foo)
                            books.append(baseName)
                        except:
                            print "reading problem",r[1],""
                        ft=foo+f.filename.split('\\')[1]
                        fo=foo+f.filename.split('\\')[0]
                        fn=fo+"/"+f.filename.split('\\')[1]
                        os.rename(fn,ft) 
                        os.rmdir(fo)
                if not booFound:
                    print "*couldnt find file"
                    
    
        else:
            print "couldnt find file  ",found
    

In [20]:
dr='/media/some/sam/download-ep/'
try:
    lst=load_obj("datas/lstepubdown")
except:
    lst=!find "$dr" -type f -iname "*.epub"
    
    save_obj(lst,"datas/lstepubdown")
len(lst)

4675

In [67]:
def cleanSe(seTerm):
    cmlst=list(set([it.lower().strip() for it in open('commonLst.dat').readlines()]))
    seTerm=''.join(fnn for fnn in seTerm if fnn in string.letters+" ")
    seTerm=' '.join(it.strip().lower() for it in seTerm.split(" ") if not any(similarity(it,b.lower())>.9 for b in cmlst)) 
    seTerm=' '.join(seTerm.split())
    return [se.strip().lower() for se in list(set(seTerm.split(' '))) if len(se)>2]
def findFile(f):  
    fp,fn=os.path.split(f)
    ext=re.findall('.+(\..+)',fn)[0]
    fn=fn.replace('_'," ").replace(ext,'').lower()
    fpr=re.findall('.*/(.+)',fp)[0].replace('_'," ").lower()
  
    se=cleanSe(fn)
    found=searchindd(se)
   
    if len(found)>0:
        #print found
        return [f,' '.join(se),found]
    else:
        ser=fn+" "+fpr
        se=cleanSe(ser)
        found=searchindd(se) 
        if len(found)>0:
        #print found
            return [f,' '.join(se),found]
        else:
            #print 'search term: ',fpr
            se=cleanSe(fpr)
            found=searchindd(se) 
            if len(found)>0:
                return [f,' '.join(se),found]
      

In [69]:
t0 = time.time()
#print lst[0]
print findFile(lst[4484])
print time.time()-t0

Search Term:  poems phantasmagoria
Search Term:  poems phantasmagoria caroll lewis
Search Term:  lewis caroll
None
91.416041851


In [ ]:

t0 = time.time()
p = Pool(6)
rmlst=p.map(findFile, lst[:500])
p.terminate()
p.close()
rmlst=filter(None,rmlst)
t1 = time.time()
total = t1-t0
print("files to delete",len(rmlst))
print('time: ',total)
print('resources: ',resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1000)

Search Term:  captive john norman gor
Search Term:  caught love her
Search Term:  assassins robin hobb apprentice
Search Term:  isaac catastrophes asimov
Search Term:  bibi mike resnick
Search Term:  stephen gods evening king
Search Term:  bibi mike resnick
Search Term:  start lauren blakely
Search Term:  isaac catastrophes asimov
Search Term:  kathryn kristine rusch assassins dagger
Search Term:  case stephen king
Search Term:  stephen gods evening king
Search Term:  larry bird niven hand
Search Term:  start lauren blakely
Search Term:  case conscience macleod ken
Search Term:  case stephen king
Search Term:  orson ender gifts scott war card
Search Term:  lauren blakely
Search Term:  quest assassins robin hobb
Search Term:  larry bird niven hand
Search Term:  case conscience macleod ken
Search Term:  cordwainer oneill smith casher planet gem
Search Term:  beautiful jamie mcguire wedding
Search Term:  john assassin norman gor
Search Term:  blakes progress nelson
Search Term:  fools dea

In [52]:
fls=zip(*rmlst)[0]
print fls,zip(*rmlst)[1]
#print zip(*rmlst)[1]
lst=filter(lambda x: x not in fls,lst)
len(lst)

('/media/some/sam/download-ep/Calibre Ebook Group Flood - Warhammer - 428 Books/Sarah Cawkwell/Bitter End (202)/Bitter End - Sarah Cawkwell.epub', '/media/some/sam/download-ep/Calibre Ebook Group Flood - Warhammer - 428 Books/Sarah Cawkwell/Cause & Effect (203)/Cause & Effect - Sarah Cawkwell.epub') ('sarah end - cawkwell (202) bitter', 'sarah & (203) - effect cawkwell cause')


4673

/home/some/repo


In [55]:
for f in fls:
    if os.path.exists(f):
        os.remove(f)

In [56]:
fls

('/media/some/sam/download-ep/Calibre Ebook Group Flood - Warhammer - 428 Books/Sarah Cawkwell/Bitter End (202)/Bitter End - Sarah Cawkwell.epub',
 '/media/some/sam/download-ep/Calibre Ebook Group Flood - Warhammer - 428 Books/Sarah Cawkwell/Cause & Effect (203)/Cause & Effect - Sarah Cawkwell.epub')

In [25]:
from multiprocessing import Process
import sharedmem
import numpy

def do_work(data, start):
    data[start] = 0;

def split_work(num):
    n = 20
    width  = n/num
    shared = sharedmem.empty(n)
    shared[:] = numpy.random.rand(1, n)[0]
    print "values are %s" % shared

    processes = [Process(target=do_work, args=(shared, i*width)) for i in xrange(num)]
    for p in processes:
        p.start()
    for p in processes:
        p.join()

    print "values are %s" % shared
    print "type is %s" % type(shared[0])

if __name__ == '__main__':
    split_work(4)

values are [ 0.42667517  0.08130225  0.18769461  0.62685297  0.91800376  0.72639408
  0.04488596  0.06138383  0.53230967  0.94864732  0.95114921  0.51271149
  0.50810097  0.04019267  0.71379624  0.42901035  0.74664958  0.58132285
  0.47031118  0.97987564]
values are [ 0.          0.08130225  0.18769461  0.62685297  0.91800376  0.
  0.04488596  0.06138383  0.53230967  0.94864732  0.          0.51271149
  0.50810097  0.04019267  0.71379624  0.          0.74664958  0.58132285
  0.47031118  0.97987564]
type is <type 'numpy.float64'>
